## Метрики регрессии: ручная реализация и сравнение с sklearn

В данном ноутбуке метрики качества регрессии реализуются вручную
на основе их математических определений, после чего результаты
сравниваются с реализациями из библиотеки `sklearn`.


In [17]:
from typing import Any, Dict, Tuple

import numpy as np
from numpy import floating

from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Подготовка данных

Для сравнения метрик используются заранее заданные массивы
истинных и предсказанных значений.

In [39]:
X, y = make_regression(n_samples=1000, n_features=10, noise=0.2, random_state=42)

### Ручной расчёт метрик

Ниже метрики MAE, MSE, RMSE, R² и MAPE вычисляются напрямую
по их математическим формулам без использования готовых функций.

Цель — показать соответствие теоретических определений
и практической реализации.

In [40]:
def solution_ms(data: Tuple[np.ndarray, np.ndarray]) -> Dict[str, np.ndarray]:
    """
    Function for training a regression model and manually calculating metrics.

    Returns:
        dict: Dictionary with metrics.

    """
    X, y = data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    line_reg = LinearRegression()
    line_reg.fit(X_train, y_train)

    y_pred = line_reg.predict(X_test)

    '''
    Mean Absolute Error (MAE)
    MAE измеряет среднюю абсолютную величину ошибки между предсказанием и фактическим значением.
    Каждая ошибка вносит вклад линейно, без усиления больших отклонений.

    Метрика хорошо интерпретируется и устойчива к выбросам по сравнению с квадратичными метриками,
    однако не различает редкие крупные ошибки и частые мелкие.
    '''
    mae = np.mean(np.abs(y_test - y_pred))

    '''
    Mean Squared Error (MSE)

    MSE вычисляет среднюю квадратичную ошибку, усиливая вклад больших отклонений
    за счёт возведения ошибки в квадрат.

    Метрика чувствительна к выбросам и часто используется в оптимизационных алгоритмах,
    поскольку обладает удобными математическими свойствами (дифференцируемость).
    '''
    mse = np.mean((y_test - y_pred)**2)

    '''
    Root Mean Squared Error (RMSE)

    RMSE является квадратным корнем из MSE и возвращает значение ошибки
    в тех же единицах измерения, что и целевая переменная.

    По смыслу RMSE сохраняет чувствительность к большим ошибкам,
    но при этом остаётся более интерпретируемой по сравнению с MSE.
    '''
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))

    '''
    Коэффициент детерминации (r2)

    r2 показывает долю дисперсии целевой переменной,
    объяснённую моделью, по сравнению с простой базовой моделью
    (предсказание среднего значения).

    Метрика не измеряет абсолютную величину ошибки и может принимать
    отрицательные значения, если модель работает хуже базового среднего.
    '''
    r2 = 1-(np.sum((y_test - y_pred)**2) / np.sum((y_test-np.mean(y_test))**2))

    '''
    Mean Absolute Percentage Error (MAPE)

    MAPE измеряет среднюю относительную ошибку в процентах
    и удобна для интерпретации, особенно в бизнес-задачах.

    Однако метрика чувствительна к малым значениям целевой переменной
    и может принимать очень большие значения при фактических значениях,
    близких к нулю.
    '''
    mape = np.mean(np.abs((y_test - y_pred) / y_test))
    return {
        "y_pred": y_pred,
        "y_test": y_test,
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "mape": mape,
        
    }

### Сравнение с реализациями sklearn

Для валидации ручных вычислений результаты сравниваются
с функциями из модуля `sklearn.metrics`.

In [37]:
def solution_sk(data: Tuple[np.ndarray, np.ndarray]) -> Dict[str, np.ndarray]:
    """
    Function for training a regression model and calculating metrics using sklearn.

    Returns:
        dict: Dictionary with metrics.

    """

    X, y = data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    line_reg = LinearRegression()
    line_reg.fit(X_train, y_train)

    y_pred = line_reg.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    return {
        "y_pred": y_pred,
        "y_test": y_test,
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "r2": r2,
        "mape": mape,
        
    }

In [32]:
data = prepare_data()
metrics_sk = solution_sk(data)
metrics_ms = solution_ms(data)

In [35]:
print(f' my own MSE: {metrics_ms["mse"]}')
print(f'sklearn MSE: {metrics_sk["mse"]}')
print(f' my own rMSE: {metrics_ms["rmse"]}')
print(f'sklearn rMSE: {metrics_sk["rmse"]}')
print(f' my own MAE: {metrics_ms["mae"]}')
print(f'sklearn MAE: {metrics_sk["mae"]}')
print(f' my own r2: {metrics_ms["r2"]}')
print(f'sklearn r2: {metrics_sk["r2"]}')
print(f' my own MAPE: {metrics_ms["mape"]}')
print(f'sklearn MAPE: {metrics_sk["mape"]}')

 my own MSE: 0.038047659641662054
sklearn MSE: 0.038047659641662054
 my own rMSE: 0.19505809299196497
sklearn rMSE: 0.19505809299196497
 my own MAE: 0.15546656260191663
sklearn MAE: 0.15546656260191663
 my own r2: 0.9999977445402571
sklearn r2: 0.9999977445402571
 my own MAPE: 0.004269556619241343
sklearn MAPE: 0.004269556619241343


### Выводы

Ручная реализация метрик полностью совпадает
с результатами, полученными с помощью `sklearn`.

Это подтверждает корректность вычислений
и помогает лучше понять свойства и различия
между метриками регрессии.
